In [1]:
!pip install numpy opencv-python pillow tqdm requests moviepy

In [16]:
!pip install tweepy

In [21]:
import os
import random
import logging
import time
import requests
from google.colab import files

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

class InstagramReelsUploader:
    def __init__(self, access_token):
        """
        Initialize the Instagram Reels uploader with the provided access token.
        """
        self.access_token = access_token
        self.api_base_url = "https://graph.facebook.com/v18.0"  # Using latest stable version
        self.instagram_account_id = None  # Initialize this attribute

        # Validate the token
        self._validate_token()

    def _validate_token(self):
        """
        Validate the access token and get user information.
        """
        try:
            # Get information about the token (me endpoint)
            response = requests.get(
                f"{self.api_base_url}/me",
                params={"access_token": self.access_token}
            )

            if response.status_code != 200:
                logging.error(f"Token validation failed: {response.json()}")
                raise ValueError("Invalid access token. Please provide a valid Instagram access token.")

            user_info = response.json()
            logging.info(f"Authenticated as user ID: {user_info.get('id')}")

            # Store user ID for later use
            self.user_id = user_info.get('id')
            # Default to using user ID if we can't find an Instagram business account
            self.instagram_account_id = self.user_id

            # Get connected Instagram Business Account ID
            response = requests.get(
                f"{self.api_base_url}/{self.user_id}/accounts",
                params={"access_token": self.access_token}
            )

            if response.status_code != 200:
                logging.warning("Failed to retrieve Instagram business accounts. Make sure the token has necessary permissions.")
                logging.info(f"Continuing with user ID: {self.user_id} as account ID")
            else:
                # Try to get Instagram business account
                accounts = response.json().get('data', [])
                if not accounts:
                    logging.warning("No Facebook pages found associated with this account.")
                    logging.info(f"Using user ID as account ID: {self.user_id}")
                else:
                    # Get the first page ID
                    page_id = accounts[0].get('id')

                    # Now get Instagram Business Account ID
                    response = requests.get(
                        f"{self.api_base_url}/{page_id}",
                        params={
                            "fields": "instagram_business_account",
                            "access_token": self.access_token
                        }
                    )

                    if response.status_code == 200 and 'instagram_business_account' in response.json():
                        self.instagram_account_id = response.json()['instagram_business_account']['id']
                        logging.info(f"Using Instagram Business Account ID: {self.instagram_account_id}")
                    else:
                        logging.warning("No Instagram Business Account found. Using main user ID.")

            # Final check to ensure we have an account ID to use
            if not self.instagram_account_id:
                self.instagram_account_id = self.user_id
                logging.info(f"Defaulting to user ID as account ID: {self.instagram_account_id}")

        except Exception as e:
            logging.error(f"Error validating access token: {e}")
            raise

    def _generate_dynamic_caption(self, reel_number):
        """
        Generate an engaging, random caption for the reel.
        """
        content_themes = [
            "daily motivation", "lifestyle tips", "fashion inspiration",
            "fitness journey", "creative expression", "travel adventures"
        ]

        viral_hooks = [
            "You won't believe this transformation! 🤯",
            "This changed my perspective forever! 😱",
            "The secret everyone needs to know... 🌟",
            "Watch until the end for the surprise! 🔍",
            "Life-changing moments caught on camera! 💥"
        ]

        call_to_actions = [
            "Follow for more content like this! 🚀",
            "Double tap if you agree! ❤️",
            "Drop your thoughts in the comments! 💬",
            "Share with someone who needs to see this! 🤝",
            "Save this for later! 📌"
        ]

        theme = random.choice(content_themes)
        hook = random.choice(viral_hooks)
        cta = random.choice(call_to_actions)

        captions = [
            f"Sharing my {theme} with you all today! {hook}\n\n"
            f"Day {reel_number}: Embracing growth and new experiences ✨\n\n"
            f"{cta}\n\n"
            "#ReelItReal #ContentCreator #LifestyleContent #InstagramReels #CreatorCommunity",

            f"Step into my world of {theme}! {hook}\n\n"
            f"Experience {reel_number}: Where passion meets purpose 🌈\n\n"
            f"{cta}\n\n"
            "#ReelsDaily #TrendAlert #ViralContent #CreativeSpace #InstaInspo",

            f"Bringing you the best {theme} content! {hook}\n\n"
            f"Episode {reel_number}: Making memories that last forever 💖\n\n"
            f"{cta}\n\n"
            "#ReelOfTheDay #TrendingNow #ContentMagic #InstagramCreator #ShareYourStory"
        ]

        return random.choice(captions)

    def _generate_hashtags(self, reel_number):
        """
        Generate a diverse set of hashtags to improve discoverability.
        """
        base_tags = [
            f"ReelSeries{reel_number}", "ReelItReal", "ContentCreator",
            "InstagramReels", "CreatorCommunity"
        ]

        theme_tags = [
            "MotivationMonday", "LifestyleTips", "FashionInspo",
            "FitnessJourney", "CreativeExpression", "TravelAdventures"
        ]

        trending_tags = [
            "ViralContent", "ReelsDaily", "TrendAlert",
            "InstaInspo", "ShareYourStory", "TrendingReels"
        ]

        return base_tags + [random.choice(theme_tags)] + random.sample(trending_tags, 3)

    def upload_reel(self, video_path, reel_number):
        """
        Upload a video as an Instagram Reel with dynamic metadata.
        """
        try:
            # Check if we have a valid account ID
            if not self.instagram_account_id:
                logging.error("No Instagram account ID available. Cannot upload.")
                return None

            # Generate caption and hashtags
            caption = self._generate_dynamic_caption(reel_number)
            hashtags = self._generate_hashtags(reel_number)

            # Format hashtags as a string and append to caption
            hashtags_str = ' '.join(f"#{tag}" for tag in hashtags)
            full_caption = f"{caption}\n\n{hashtags_str}"

            logging.info(f"Uploading video: {video_path} to account: {self.instagram_account_id}")

            # Check if file exists and is readable
            if not os.path.exists(video_path):
                logging.error(f"Video file not found: {video_path}")
                return None

            file_size = os.path.getsize(video_path)
            logging.info(f"Video file size: {file_size} bytes")

            # Step 1: Create a container for the media upload
            logging.info("Creating media container...")

            # For Instagram, we need to upload the video first to a container
            # We'll use direct video URL approach for simplicity in this example
            container_params = {
                "access_token": self.access_token,
                "media_type": "REELS",
                "caption": full_caption,
                "share_to_feed": "true"
            }

            # Check if file is accessible
            try:
                with open(video_path, 'rb') as test_file:
                    pass
                logging.info("File is accessible for reading")
            except Exception as e:
                logging.error(f"Cannot access video file: {e}")
                return None

            # Alternative approach: Use Facebook's video upload API directly
            # This bypasses some of the Instagram-specific complexity
            with open(video_path, 'rb') as video_file:
                # Create a post directly to the user's feed with the video
                files = {'source': (os.path.basename(video_path), video_file, 'video/mp4')}

                upload_response = requests.post(
                    f"{self.api_base_url}/{self.instagram_account_id}/videos",
                    params={
                        "access_token": self.access_token,
                        "description": full_caption,
                    },
                    files=files
                )

                if upload_response.status_code != 200:
                    logging.error(f"Failed to upload video: {upload_response.text}")

                    # Try alternative method if first approach fails
                    logging.info("Trying alternative upload method...")
                    video_file.seek(0)  # Reset file pointer

                    # Create container first
                    container_response = requests.post(
                        f"{self.api_base_url}/{self.instagram_account_id}/media",
                        params={
                            "access_token": self.access_token,
                            "media_type": "VIDEO",
                            "caption": full_caption
                        }
                    )

                    if container_response.status_code != 200:
                        logging.error(f"Failed to create media container: {container_response.text}")
                        return None

                    container_id = container_response.json().get('id')
                    logging.info(f"Created container: {container_id}")

                    # Now try to publish
                    publish_response = requests.post(
                        f"{self.api_base_url}/{self.instagram_account_id}/media_publish",
                        params={
                            "access_token": self.access_token,
                            "creation_id": container_id
                        }
                    )

                    if publish_response.status_code != 200:
                        logging.error(f"Failed to publish media: {publish_response.text}")
                        return None

                    media_id = publish_response.json().get('id')
                    logging.info(f"Successfully published media: {media_id}")
                    return media_id

                # If first approach succeeded
                media_id = upload_response.json().get('id')
                logging.info(f"Video uploaded successfully! Media ID: {media_id}")
                return media_id

        except Exception as e:
            logging.error(f"An error occurred during upload: {e}")
            return None

        finally:
            # Add a delay to respect API rate limits
            time.sleep(2)


def main():
    """
    Upload all generated videos to Instagram Reels
    """
    # Access token provided
    ACCESS_TOKEN = "EAAMLekxxGpcBOZCFAC8b8kcxg2LDkVpJB7LCjEoF7dhOTlfpmqWZB6uAzJtxpzCm0DnHeA0hG6umaZCINb9dLqT4Lq2oOCTUii7oeh5taZBYxm8PXD3KzH4HFHNFAmRoTfu8EdnhzxQXI1hWYQEay6f1LZBpeDTkhvfIbHk6TliZA9kCXdjE9rXzVMf2gRF5rP"

    # Ensure you have the videos directory mounted or uploaded
    videos_dir = "/content/Video/final_videos/"

    # Create the directory if it doesn't exist
    os.makedirs(videos_dir, exist_ok=True)

    # Prompt to upload videos if directory is empty
    if not os.listdir(videos_dir):
        print("Please upload your video files to the /content/Video/final_videos/ directory.")
        uploaded_files = files.upload()

        # Move uploaded files to the correct directory
        for filename in uploaded_files:
            os.rename(filename, os.path.join(videos_dir, filename))

    # Find video files
    video_files = [
        os.path.join(videos_dir, f)
        for f in os.listdir(videos_dir)
        if f.endswith('.mp4')
    ]

    # Validate video files
    if not video_files:
        raise ValueError("No video files found to upload.")

    # Print found video files
    logging.info(f"Found {len(video_files)} video files to upload:")
    for video in video_files:
        logging.info(f" - {video}")

    try:
        # Initialize uploader with additional error handling
        logging.info("Initializing Instagram Reels Uploader...")
        uploader = InstagramReelsUploader(ACCESS_TOKEN)

        # Upload each video
        for video_path in video_files:
            # Extract reel number from filename
            reel_number = os.path.splitext(os.path.basename(video_path))[0].split('_')[-1]
            logging.info(f"Uploading video {video_path} as reel #{reel_number}")

            try:
                result = uploader.upload_reel(video_path, reel_number)
                if result:
                    logging.info(f"Successfully uploaded reel #{reel_number}")
                else:
                    logging.error(f"Failed to upload reel #{reel_number}")

                # Add configurable delay between uploads
                delay = 5  # Reduced from 30 to 5 seconds for testing
                logging.info(f"Waiting {delay} seconds before next upload...")
                time.sleep(delay)

            except KeyboardInterrupt:
                logging.info("Upload process interrupted by user. Exiting...")
                break

            except Exception as e:
                logging.error(f"Failed to upload {video_path}: {e}")
                logging.info("Continuing with next video...")
                continue

    except KeyboardInterrupt:
        logging.info("Process interrupted by user. Exiting...")
    except Exception as e:
        logging.error(f"Fatal error in main process: {e}")


if __name__ == '__main__':
    main()

ERROR:root:Failed to upload video: {"error":{"message":"(#100) No permission to publish the video","type":"OAuthException","code":100,"fbtrace_id":"AyE4tV059kQ_HPQK2hOS5cs"}}
ERROR:root:Failed to create media container: {"error":{"message":"Unsupported post request. Object with ID '122102099690790307' does not exist, cannot be loaded due to missing permissions, or does not support this operation. Please read the Graph API documentation at https:\/\/developers.facebook.com\/docs\/graph-api","type":"GraphMethodException","code":100,"error_subcode":33,"fbtrace_id":"ARHL7R1_J5fqS0bGRSlOjlo"}}
ERROR:root:Failed to upload reel #4
ERROR:root:Failed to upload video: {"error":{"message":"(#100) No permission to publish the video","type":"OAuthException","code":100,"fbtrace_id":"A_iVakjNCCF8M3lpG3fXj_C"}}
ERROR:root:Failed to create media container: {"error":{"message":"Unsupported post request. Object with ID '122102099690790307' does not exist, cannot be loaded due to missing permissions, or d